# Problema 23 - Application Incident Prediction

## Pré-processamento

### Carregando os dados e realizando limpeza inicial. ###

In [21]:
# Importando bibliotecas necessárias
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Carregando o dataset
df = pd.read_csv('app_incident_report.csv')

# Removendo a coluna 'incident_duration' conforme especificado no enunciado
df = df.drop('incident_duration', axis=1)

print("Dataset carregado. Dimensões:", df.shape)
df.head()

Dataset carregado. Dimensões: (54745, 8)


,app_name,response_time,error_rate,cpu_usage,memory_usage,disk_space,active_users,downtime
0,NIP,22,61,32,42,52,12000,1
1,NIP,21,63,31,41,51,11000,0
2,NIP,20,98,30,40,50,10000,0
3,NIP,19,15,29,39,49,9000,0
4,NIP,18,67,28,38,48,8000,0


### Analisar se é necessário remover a coluna `app_name`

Olhando os dados por cima notei que o app_name é sempre o mesmo, então nesse passo vamos verificar quantos valores únicos existem na coluna `app_name`. Se houver apenas um, a coluna é constante e não contribui com informação para o modelo, podendo ser removida com segurança.

In [22]:
# Verificando os valores únicos em 'app_name'
unique_apps = df['app_name'].nunique()
app_counts = df['app_name'].value_counts()

print(f"Número de valores únicos em 'app_name': {unique_apps}")
print("\nContagem de cada valor:")
print(app_counts)

if unique_apps == 1:
    print("\nConclusão: A coluna 'app_name' possui um valor constante e será removida.")

Número de valores únicos em 'app_name': 1

Contagem de cada valor:
app_name
NIP    54745
Name: count, dtype: int64

Conclusão: A coluna 'app_name' possui um valor constante e será removida.


### Separação de Características (X) e Alvo (y)

Separando o dataset em Características (features) e Alvo (target):
* **X**: Todas as colunas de características que usaremos para treinar o modelo.
* **y**: A coluna alvo que queremos prever (`downtime`).

In [23]:
# Separando as features (X) e o target (y)
X = df.drop(['downtime', 'app_name'], axis=1)
y = df['downtime']

print("Shape de X (features):", X.shape)
print("Shape de y (target):", y.shape)

Shape de X (features): (54745, 6)
Shape de y (target): (54745,)


### Análise da Variável Alvo (`downtime`)

Pela análise visual foi possível ver que a distribuição da variável `downtime` é completamente
desbalanceada. Esse desbalanceamento pode ser preocupante na hora da divisão dos dados.

In [24]:
# Calculando a contagem e a proporção de cada classe
downtime_counts = y.value_counts()
downtime_percentage = y.value_counts(normalize=True) * 100

print("Contagem de cada classe:")
print(downtime_counts)
print("\nPorcentagem de cada classe:")
print(downtime_percentage)

Contagem de cada classe:
downtime
0    54197
1      548
Name: count, dtype: int64

Porcentagem de cada classe:
downtime
0    98.998995
1     1.001005
Name: proportion, dtype: float64


### Divisão em Dados de Treino e Teste

Será separado 20% dos dados para teste.

Como demonstrado na análise acima, a classe `downtime=1` é bem rara (correspondendo a aproximadamente **1%** dos dados). Por isso, é necessário usar o parâmetro `stratify=y`. Essa estratificação garante que a proporção de downtime seja mantida tanto para o conjunto de treino quanto para o de teste.

In [25]:
# Dividindo os dados com test_size=0.2 (20%) e estratificação
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("--- Shapes após a divisão ---")
print("X_train:", X_train.shape)
print("X_test:", X_test.shape)
print("y_train:", y_train.shape)
print("y_test:", y_test.shape)

print("\n--- Verificação da estratificação ---")
print("Proporção de downtime no y_train:", y_train.value_counts(normalize=True).values[1])
print("Proporção de downtime no y_test:", y_test.value_counts(normalize=True).values[1])

--- Shapes após a divisão ---
X_train: (43796, 6)
X_test: (10949, 6)
y_train: (43796,)
y_test: (10949,)

--- Verificação da estratificação ---
Proporção de downtime no y_train: 0.010000913325417846
Proporção de downtime no y_test: 0.010046579596310166


### Escalonamento das Características

As colunas numéricas têm escalas muito diferentes. Por exemplo, active_users vai de 0 a 12.000, enquanto cpu_usage vai de 20 a 32.
Portanto, será utilizado o `StandardScaler` para padronizar os dados (média 0 e desvio padrão 1).
O `StandardScaler` é "treinado" (`fit`) apenas com os dados de treino para evitar vazamento de informação dos dados de teste. Depois, ele é usado para transformar (`transform`) ambos os conjuntos.

In [26]:
# Criando o objeto scaler
scaler = StandardScaler()

# Treinando o scaler com os dados de treino e transformando-os
X_train_scaled = scaler.fit_transform(X_train)

# Aplicando a mesma transformação aos dados de teste
X_test_scaled = scaler.transform(X_test)

# Convertendo de volta para DataFrame para visualização (opcional)
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)

print("Amostra dos dados de treino após o escalonamento:")
X_train_scaled_df.head()

Amostra dos dados de treino após o escalonamento:


,response_time,error_rate,cpu_usage,memory_usage,disk_space,active_users
0,0.574671,-0.578807,0.574671,0.574671,0.574671,0.574671
1,1.147714,0.596261,1.147714,1.147714,1.147714,1.147714
2,-1.717498,-1.131780,-1.717498,-1.717498,-1.717498,-1.717498
3,0.001629,-0.820733,0.001629,0.001629,0.001629,0.001629
4,0.001629,-1.028098,0.001629,0.001629,0.001629,0.001629


### Conclusão do Pré-processamento

Os seguintes conjuntos de dados estão preparados:

* `X_train_scaled`, `y_train`: Para treinar os modelos.
* `X_test_scaled`, `y_test`: Para testar os modelos e gerar os resultados finais.

## Naive Bayes

Para o Bayes Ingênuo será utilizado o `GaussianNB` do scikit-learn.

In [27]:
from sklearn.naive_bayes import GaussianNB

nb_model = GaussianNB()

# Treinando o modelo com os dados de treino (já escalonados)
nb_model.fit(X_train_scaled, y_train)

# Fazendo predições no conjunto de teste
y_pred_nb = nb_model.predict(X_test_scaled)

print("Modelo Naive Bayes treinado com sucesso!")
print(f"Foram feitas {len(y_pred_nb)} predições no conjunto de teste.")

Modelo Naive Bayes treinado com sucesso!
Foram feitas 10949 predições no conjunto de teste.


### Avaliação do Modelo Naive Bayes

Para avaliar o desempenho, será gerado a matriz de confusão e a medida F1.

In [28]:
# Importando as métricas de avaliação
from sklearn.metrics import confusion_matrix, f1_score

# Calculando a Matriz de Confusão
cm_nb = confusion_matrix(y_test, y_pred_nb)

# Calculando a Medida F1
f1_nb = f1_score(y_test, y_pred_nb)

print("--- Resultados do Modelo Naive Bayes ---")
print("\nMatriz de Confusão:")
print(cm_nb)
print(f"\nMedida F1 (F1 Score): {f1_nb:.4f}")

--- Resultados do Modelo Naive Bayes ---

Matriz de Confusão:
[[10839     0]
 [  110     0]]

Medida F1 (F1 Score): 0.0000


### Visualização da Matriz de Confusão

Para facilitlar a compreensão abaixo está o plot do gráfico da matriz de confusão com Plotly.

In [29]:
import plotly.express as px

labels = ['Sem Downtime', 'Com Downtime']

fig_cm_nb = px.imshow(
    cm_nb,
    text_auto=True, 
    labels=dict(x="Valor Predito", y="Valor Real", color="Contagem"),
    x=labels,
    y=labels,
    color_continuous_scale='GnBu'
)

fig_cm_nb.update_layout(
    title_text='<b>Matriz de Confusão - Naive Bayes</b>',
    title_x=0.5,
    width=550,
    height=500,
    template='plotly_white',
    font=dict(
        family="Arial, sans-serif",
        size=12,
        color="black"
    ),
    xaxis_title="<b>Valor Predito</b>",
    yaxis_title="<b>Valor Real</b>"
)

fig_cm_nb.update_traces(textfont_size=16)

fig_cm_nb.show()

### Análise dos Resultados - Naive Bayes

#### Matriz de Confusão

* **Verdadeiros Negativos (10839):** O modelo é excelente em prever "Sem Downtime" quando o sistema está normal.
* **Falsos Positivos (0):** O modelo foi extremamente conservador e nunca gerou um "alarme falso", ou seja, nunca previu um downtime que não aconteceu.
* **Falsos Negativos (110):** Este é o grande problema. Em todos os 110 casos em que um downtime realmente ocorreu, o modelo falhou e previu "Sem Downtime".
* **Verdadeiros Positivos (0):** Consequentemente, o modelo não foi capaz de identificar corretamente um único incidente de downtime.

#### Medida F1?

O fato da medida F1 ser 0.0 é consequência de um modelo simples treinado com dados altamente desbalanceados, onde
o "Sem Downtime" representa 99% dos casos. Então o modelo aprendeu que a maneira mais fácil de obter um alto acerto era basicamente
prever a classe que mais aparecer.

## Perceptron Multicamadas (MLP)

Será utilizado a implementação `MLPClassifier` do scikit-learn.

Conforme solicitado no enunciado:

* **Arquitetura da Rede:**
    * `hidden_layer_sizes=(100,)`: Esta é a arquitetura da rede. Será usado uma única camada oculta contendo 100 neurônios.

* **Parâmetros de Treinamento:**
    * `activation='relu'`: A função de ativação será a Unidade Linear Retificada (ReLU), que é a mais comum e eficiente atualmente.
    * `solver='adam'`: O otimizador utilizado para minimizar a função de erro. 'Adam' é eficiente e se adapta bem à maioria dos problemas.
    * `max_iter=500`: O número máximo de épocas de treinamento.
    * `early_stopping=True`: Uma técnica para evitar sobreajuste. O treinamento para se o desempenho em um conjunto de validação (separado automaticamente dos dados de treino) não melhorar por um número definido de épocas.
    * `n_iter_no_change=10`: O número de épocas a esperar sem melhora antes de parar o treinamento.
    * `random_state=42`: Garante que os resultados sejam reprodutíveis.

In [ ]:
from sklearn.neural_network import MLPClassifier

mlp_model = MLPClassifier(
    hidden_layer_sizes=(100,),
    activation='relu',
    solver='adam',
    max_iter=500,
    early_stopping=True,
    n_iter_no_change=10,
    random_state=42
)

#Treinando o modelo com os dados de treino escalonados
mlp_model.fit(X_train_scaled, y_train)

#Fazendo predições no conjunto de teste
y_pred_mlp = mlp_model.predict(X_test_scaled)

print("Modelo MLP treinado com sucesso!")

TypeError: MLPClassifier.__init__() got an unexpected keyword argument 'class_weight'

### Avaliação do Modelo MLP

Para avaliar o resultado será gerado a matriz de confusão e a Medida F1.

In [ ]:
cm_mlp = confusion_matrix(y_test, y_pred_mlp)

f1_mlp = f1_score(y_test, y_pred_mlp)

print("--- Resultados do Modelo MLP ---")
print("\nMatriz de Confusão:")
print(cm_mlp)
print(f"\nMedida F1 (F1 Score): {f1_mlp:.4f}")

--- Resultados do Modelo MLP ---

Matriz de Confusão:
[[10839     0]
 [  110     0]]

Medida F1 (F1 Score): 0.0000


### Visualização da Matriz de Confusão

Para facilitlar a compreensão abaixo está o plot do gráfico da matriz de confusão com Plotly.

In [ ]:
# Visualização da Matriz de Confusão com Plotly
fig_cm_mlp = px.imshow(
    cm_mlp,
    text_auto=True,
    labels=dict(x="Valor Predito", y="Valor Real", color="Contagem"),
    x=labels,
    y=labels,
    color_continuous_scale='Greens'
)

fig_cm_mlp.update_layout(
    title_text='<b>Matriz de Confusão - MLP</b>',
    title_x=0.5,
    width=550,
    height=500,
    template='plotly_white',
    font=dict(family="Arial, sans-serif", size=12, color="black"),
    xaxis_title="<b>Valor Predito</b>",
    yaxis_title="<b>Valor Real</b>"
)
fig_cm_mlp.update_traces(textfont_size=16)
fig_cm_mlp.show()

## Comparação Final dos Modelos

In [ ]:
final_results = {
    'Modelo': ['Naive Bayes', 'MLP'],
    'Medida F1': [f1_nb, f1_mlp]
}
results_df = pd.DataFrame(final_results)

print("--- Tabela Final de Resultados ---")
print(results_df.round(4))

--- Tabela Final de Resultados ---
        Modelo  Medida F1
0  Naive Bayes        0.0
1          MLP        0.0


## Análise e Conclusão Final

A avaliação dos modelos demonstrou que ambos foram incapazes de gerar um classificador funcional para o problema, resultando em uma Medida F1 de 0.0 para os dois.

A causa para este desempenho foi o grande desbalanceamento de classes no dataset, que levou ambos os algoritmos a preverem invariavelmente a classe majoritária ("Sem Downtime").

Percebi que seria possível usar técnicas como o SMOTE para mitigar o desbalanceamento e gerar um modelo funcional. Contudo, não segui com essa abordagem, pois notei que ela não estava nas etapas de pré-processamento do enunciado. Isso me levou a concluir que este resultado era, na verdade, o objetivo de aprendizado do trabalho: mostrar na prática o desafio de lidar com dados desbalanceados e a importância de uma métrica como a Medida F1, que expôs o problema claramente.